In [1]:
# importing the moduels
import pandas as pd
import requests
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt

In [2]:
# creating crime data frame
crime_df = pd.read_csv("Resources/Crime_Data_from_2020_to_present.csv")
crime_df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [ ]:
# Step 2: Crime Type Analysis - Rekha
# Plot the distribution of different crime types as  bar graph
crime_counts = crime_df['Crm Cd Desc'].value_counts()
crime_counts.plot(kind='bar', figsize=(10, 6))
plt.title('Distribution of Crime Types')
plt.xlabel('Crm Cd Desc')
plt.ylabel('Number of Incidents')
plt.show()

In [ ]:
#Demographics(Jesus) - Clean Victim Sex and Age column, create demographic dataframe
demo_crime_df = crime_df.copy()
demo_crime_df.loc[crime_df['Vict Age'] < 0] = 0
demo_crime_df['Vict Sex'] = demo_crime_df['Vict Sex'].replace(['H', '-', 0, 'X' ], 'Unknown')
demo_crime_df['Vict Descent'] = demo_crime_df['Vict Descent'].replace(['-', 0], 'O')

In [ ]:
#Demographics(Jesus) - Create age group columns
size_bins = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150]
labels = ['Under 10', '10-19', '20-29', '30-39', '40-49', '50-59','60-69','70-79','80-89','90-99','Over 100']
demo_crime_df["age range"] = pd.cut(demo_crime_df['Vict Age'],size_bins,labels=labels,right=False)


In [ ]:
#Demographics(Jesus) - Age group count check
age_range_count =  demo_crime_df['age range'].value_counts().sort_index() 
age_range_count

In [ ]:
#Demographics(Jesus) -  Generate a bar plot showing the total number of victms for each age group
# Plot the bar chart
count_chart = age_range_count.plot(kind='bar', figsize=(6,6))
count_chart.set_xlabel("Age groups")
count_chart.set_ylabel("Number of Victims")
count_chart.set_title("Number of Victims by Age Group")
plt.show()

In [ ]:
#Demographics(Jesus) - Pie Chart showing Percentage of Victims by Gender
# Get the unique victims with their gender
gender_count = demo_crime_df.groupby('Vict Sex')['DR_NO'].nunique().sort_values(ascending=False)

# Plot the pie chart
gender_count.plot(kind='pie',autopct="%1.1f%%")
plt.ylabel("Percentage of Victims by Gender")
plt.show()

In [ ]:
#Demographics(Jesus) -  Generate a bar plot showing the total number of victms for each age group by Gender
age_range_count =  demo_crime_df.groupby(['age range', 'Vict Sex']).size().reset_index(name='count') 

pivot_df = age_range_count.pivot(index='age range', columns='Vict Sex', values='count')

# Plot the bar chart
pivot_df.plot(kind='bar', figsize=(10, 6))
plt.xlabel("Age groups")
plt.ylabel("Number of Victims")
plt.title("Number of Victims by Age Group and Gender")
plt.show()

In [ ]:
#Demographics(Jesus) - Define the mapping
replacement_dict = {
    'A': 'Asian',
    'B': 'Black',
    'C': 'Asian',
    'D': 'Asian',
    'F': 'Asian',
    'G': 'Other',
    'H': 'Hispanic/Latino',
    'I': 'Other',
    'J': 'Asian',
    'K': 'Asian',
    'L': 'Asian',
    'O': 'Other',
    'P': 'Other',
    'S': 'Other',
    'U': 'Other',
    'V': 'Asian',
    'W': 'White',
    'X': 'Unknown',
    'Z': 'Asian'
}
demo_crime_df['Vict Descent'] = demo_crime_df['Vict Descent'].replace(replacement_dict)


In [ ]:
#Demographics(Jesus) -  Descent count check
Vict_Descent_count =  demo_crime_df.groupby('Vict Descent').size().reset_index(name='count')
Vict_Descent_count['percentage'] = (Vict_Descent_count['count'] / Vict_Descent_count['count'].sum()) * 100
Vict_Descent_count.sort_values(by='percentage', inplace=True, ascending=False)
Vict_Descent_count

In [ ]:
#Demographics(Jesus) -  Generate a bar plot showing the total number of victms by Ethnicity
# Plot the bar chart
count_chart = Vict_Descent_count.plot(kind='bar', x='Vict Descent', figsize=(6,6))
count_chart.set_xlabel("Ethnic Groups")
count_chart.set_ylabel("Number of victims")
plt.legend().set_visible(False)
plt.title("Number of Victims by Ethnicity")
plt.show()

In [ ]:
#Demographics(Jesus) -  Generate a bar plot showing the total number of victms by Age Group and  Ethnicity
Vict_Descent_count =  demo_crime_df.groupby(['age range','Vict Descent']).size().reset_index(name='count')
pivot_df = Vict_Descent_count.pivot(index='age range', columns='Vict Descent', values='count')
# Plot the bar chart
pivot_df.plot(kind='bar', figsize=(10, 6))
plt.title("Number of Victims by Age Group and Ethnicity")
plt.show()

In [ ]:
print("-----------------------")
print("Carl's code starts here")
print("-----------------------")

In [ ]:
# 

In [ ]:
print("----------------------")
print("Carl's code ends here.")
print("----------------------")